In [2]:
import numpy as np
import pandas as pd
import MySQLdb

In [3]:
# too big too load in python
# LeaveTime_DB = pd.read_csv("D:/datasets/Dublin_bus/data/rt_leavetimes_DB_2018.txt", sep=';')
# Trips_DB = pd.read_csv("D:/datasets/Dublin_bus/data/rt_trips_DB_2018.txt")
# Vehicles_DB = pd.read_csv("D:/datasets/Dublin_bus/data/rt_vehicles_DB_2018.txt")

# Import data and connect to database

In [4]:
# Connect to mysql
cxn = MySQLdb.connect(user='user', password='nUT8+~nYRS/9-i$')
cur = cxn.cursor()

In [5]:
def GetData(cur, query):
    # cur.execute('select * from busduck.leavetimes_01 LIMIT 0,10;')
    cur.execute(query)
    rows = cur.fetchall()
    return    pd.DataFrame(rows, columns=[i[0] for i in cur.description])

In [208]:
weather_DF = pd.read_csv("D:/datasets/Dublin_bus/2018_daily.csv")

In [209]:
weather_DF.tail()

,date,time,temp,humid,windspeed,condition
17142,2018/12/31,9:30 PM,48,82,15,Mostly Cloudy
17143,2018/12/31,10:00 PM,48,82,12,Mostly Cloudy
17144,2018/12/31,10:30 PM,50,76,13,Mostly Cloudy
17145,2018/12/31,11:00 PM,48,82,13,Mostly Cloudy
17146,2018/12/31,11:30 PM,48,82,13,Mostly Cloudy


## Convert column types

In [211]:
# convert to datetime
def date_to_datetime(x):
    """convert seconds to timedelta"""
    return datetime.datetime.strptime(x, "%Y/%m/%d")

In [212]:
# convert 24 hours formate
def time12_to_time24(x):
    """convert seconds to timedelta"""
    return datetime.datetime.strptime(x, "%I:%M %p").time()

In [213]:
weather_DF["date"] = weather_DF["date"].apply(date_to_datetime)
weather_DF["time"] = weather_DF["time"].apply(time12_to_time24)

In [215]:
weather_DF.head()

,date,time,temp,humid,windspeed,condition
0,2018-01-01,00:00:00,39,75,16,Partly Cloudy
1,2018-01-01,00:30:00,37,81,14,Partly Cloudy
2,2018-01-01,01:00:00,39,87,29,Light Rain Shower / Windy
3,2018-01-01,01:30:00,41,81,31,Mostly Cloudy / Windy
4,2018-01-01,02:00:00,39,87,26,Light Rain Shower / Windy


In [202]:
weather_DF.dtypes

date         datetime64[ns]
time                 object
temp                  int64
humid                 int64
windspeed             int64
condition            object
dtype: object

# Select whole year data of line:17 direction:1

In [13]:
query = """
SELECT * FROM busduck.rt_leavetimes_db where TRIPID in 
(SELECT TRIPID FROM busduck.rt_trips_db where LINEID="17" and DIRECTION = 1);
"""
DF = GetData(cur, query)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,2018-01-01,5968763,1,1380,50700,50700,50714,50714,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
1,DB,2018-01-01,5968763,2,1381,50726,50726,50731,50731,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
2,DB,2018-01-01,5968763,3,1406,50773,50773,50787,50817,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
3,DB,2018-01-01,5968763,4,1407,50816,50816,50871,50871,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
4,DB,2018-01-01,5968763,5,1409,50855,50855,50893,50893,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582002,DB,2018-12-31,8591872,65,467,65061,65061,65397,65397,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582003,DB,2018-12-31,8591872,66,468,65096,65096,65414,65414,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582004,DB,2018-12-31,8591872,67,3032,65134,65134,65521,65536,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582005,DB,2018-12-31,8591872,68,3033,65180,65180,65600,65600,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,


In [15]:
DF.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,2018-01-01,5968763,1,1380,50700,50700,50714,50714,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
1,DB,2018-01-01,5968763,2,1381,50726,50726,50731,50731,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
2,DB,2018-01-01,5968763,3,1406,50773,50773,50787,50817,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
3,DB,2018-01-01,5968763,4,1407,50816,50816,50871,50871,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,
4,DB,2018-01-01,5968763,5,1409,50855,50855,50893,50893,2406881,0,0,0,0,0,0,2018-01-08 17:21:10,


In [14]:
DF.tail(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
582002,DB,2018-12-31,8591872,65,467,65061,65061,65397,65397,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582003,DB,2018-12-31,8591872,66,468,65096,65096,65414,65414,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582004,DB,2018-12-31,8591872,67,3032,65134,65134,65521,65536,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582005,DB,2018-12-31,8591872,68,3033,65180,65180,65600,65600,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,
582006,DB,2018-12-31,8591872,69,3085,65227,65227,65623,65623,2534852,0,0,0,0,0,0,2019-01-16 18:27:21,


# Drop useless columns and duplicate rows and check null

In [46]:
unused_cols = ["DATASOURCE", "VEHICLEID", "PASSENGERS", "PASSENGERSIN", "PASSENGERSOUT",
              "DISTANCE", "SUPPRESSED", "JUSTIFICATIONID","LASTUPDATE","NOTE", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP"]
DF_droped = DF.drop(unused_cols, axis=1)

In [47]:
DF_droped

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP
0,2018-01-01,5968763,1,1380,50714,50714
1,2018-01-01,5968763,2,1381,50731,50731
2,2018-01-01,5968763,3,1406,50787,50817
3,2018-01-01,5968763,4,1407,50871,50871
4,2018-01-01,5968763,5,1409,50893,50893
...,...,...,...,...,...,...
582002,2018-12-31,8591872,65,467,65397,65397
582003,2018-12-31,8591872,66,468,65414,65414
582004,2018-12-31,8591872,67,3032,65521,65536
582005,2018-12-31,8591872,68,3033,65600,65600


In [48]:
sum(DF_droped.duplicated(keep=False))

0

It shows no duplicate rows

In [49]:
DF_droped.isnull().sum()

DAYOFSERVICE      0
TRIPID            0
PROGRNUMBER       0
STOPPOINTID       0
ACTUALTIME_ARR    0
ACTUALTIME_DEP    0
dtype: int64

It shows no null values

In [50]:
DF_droped.dtypes

DAYOFSERVICE      datetime64[ns]
TRIPID                    object
PROGRNUMBER                int64
STOPPOINTID               object
ACTUALTIME_ARR             int64
ACTUALTIME_DEP             int64
dtype: object

# Create Customized Columns 

## Add 'ARR' and 'DEP' converted to time

In [113]:
import datetime

datetime.timedelta(seconds=50700)

In [54]:
def seconds_to_timedelta(x):
    """convert seconds to timedelta"""
    return datetime.timedelta(seconds=x)

In [58]:
DF_droped["ACTUALTIME_ARR_DATETIME"] = DF_droped["DAYOFSERVICE"] + DF_droped["ACTUALTIME_ARR"].apply(seconds_to_timedelta)
DF_droped["ACTUALTIME_DEP_DATETIME"] = DF_droped["DAYOFSERVICE"] + DF_droped["ACTUALTIME_DEP"].apply(seconds_to_timedelta)

In [108]:
DF_droped["WEEK_DAY"] = DF_droped["DAYOFSERVICE"].dt.dayofweek

In [109]:
# WEEKDAY from 0 to 6
DF_droped.tail()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_ARR_DATETIME,ACTUALTIME_DEP_DATETIME,WEEK_DAY
582002,2018-12-31,8591872,65,467,65397,65397,2018-12-31 18:09:57,2018-12-31 18:09:57,0
582003,2018-12-31,8591872,66,468,65414,65414,2018-12-31 18:10:14,2018-12-31 18:10:14,0
582004,2018-12-31,8591872,67,3032,65521,65536,2018-12-31 18:12:01,2018-12-31 18:12:16,0
582005,2018-12-31,8591872,68,3033,65600,65600,2018-12-31 18:13:20,2018-12-31 18:13:20,0
582006,2018-12-31,8591872,69,3085,65623,65623,2018-12-31 18:13:43,2018-12-31 18:13:43,0


## Devided into periods according to 'ACTUALTIME_ARR_DATETIME'

if time in 2:00 ~ 2:30, the period would be 2:00 <br>
if time in 2:30 ~ 3:00, the period would be 2:30

In [157]:
def time_to_periods(x):
    """convert time to periods defined"""
    if x.minute < 30:
        m = 0
    else:
        m = 30
    return datetime.time(x.hour,m)

In [158]:
ts = pd.Series(1,index=DF_droped["ACTUALTIME_ARR_DATETIME"])

In [159]:
DF_droped["PERIOD"] = DF_droped["ACTUALTIME_ARR_DATETIME"].apply(time_to_periods)

In [160]:
DF_droped["DAYOFSERVICE"][0]

Timestamp('2018-01-01 00:00:00')

## Merge dataframes 'DF_droped" and "weather_DF"

Use left outer join to merge them based on : left_on=['DAYOFSERVICE','PERIOD'],  right_on=['date','time']

In [230]:
DF_merged = pd.merge(DF_droped, weather_DF, left_on=['DAYOFSERVICE','PERIOD'], \
         right_on=['date','time'], how='left').drop(['date','time'], axis=1)

In [231]:
DF_merged.to_csv("DF_merged.csv")

In [232]:
DF_merged.head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_ARR_DATETIME,ACTUALTIME_DEP_DATETIME,WEEK_DAY,PERIOD,temp,humid,windspeed,condition
0,2018-01-01,5968763,1,1380,50714,50714,2018-01-01 14:05:14,2018-01-01 14:05:14,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
1,2018-01-01,5968763,2,1381,50731,50731,2018-01-01 14:05:31,2018-01-01 14:05:31,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
2,2018-01-01,5968763,3,1406,50787,50817,2018-01-01 14:06:27,2018-01-01 14:06:57,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
3,2018-01-01,5968763,4,1407,50871,50871,2018-01-01 14:07:51,2018-01-01 14:07:51,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
4,2018-01-01,5968763,5,1409,50893,50893,2018-01-01 14:08:13,2018-01-01 14:08:13,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy


In [239]:

criteria = "PERIOD == datetime.time(14, 0) and WEEK_DAY==0"
DF_merged.query(criteria)
# DF_droped.loc[DF_droped.eval(criteria)]

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_ARR_DATETIME,ACTUALTIME_DEP_DATETIME,WEEK_DAY,PERIOD,temp,humid,windspeed,condition
0,2018-01-01,5968763,1,1380,50714,50714,2018-01-01 14:05:14,2018-01-01 14:05:14,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
1,2018-01-01,5968763,2,1381,50731,50731,2018-01-01 14:05:31,2018-01-01 14:05:31,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
2,2018-01-01,5968763,3,1406,50787,50817,2018-01-01 14:06:27,2018-01-01 14:06:57,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
3,2018-01-01,5968763,4,1407,50871,50871,2018-01-01 14:07:51,2018-01-01 14:07:51,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
4,2018-01-01,5968763,5,1409,50893,50893,2018-01-01 14:08:13,2018-01-01 14:08:13,0,14:00:00,45.0,81.0,29.0,Light Rain Shower / Windy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581852,2018-12-31,8591869,43,2893,51999,51999,2018-12-31 14:26:39,2018-12-31 14:26:39,0,14:00:00,50.0,71.0,9.0,Mostly Cloudy
581853,2018-12-31,8591869,44,2894,52033,52033,2018-12-31 14:27:13,2018-12-31 14:27:13,0,14:00:00,50.0,71.0,9.0,Mostly Cloudy
581854,2018-12-31,8591869,45,2895,52054,52054,2018-12-31 14:27:34,2018-12-31 14:27:34,0,14:00:00,50.0,71.0,9.0,Mostly Cloudy
581855,2018-12-31,8591869,46,3012,52096,52096,2018-12-31 14:28:16,2018-12-31 14:28:16,0,14:00:00,50.0,71.0,9.0,Mostly Cloudy


In [149]:
criteria = "PERIOD == datetime.time(14, 0) and DAYOFSERVICE == Timestamp('2018-01-01 00:00:00')"
DF_droped.eval(criteria)

0          True
1          True
2          True
3          True
4          True
          ...  
582002    False
582003    False
582004    False
582005    False
582006    False
Length: 582007, dtype: bool

In [24]:
trips_l = DF_droped['TRIPID'].unique()
len(trips_l)

2704

In [40]:
DF_droped.query("TRIPID == '{}'".format(trips_l[0]))

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,2018-01-01,5968763,1,1380,50700,50700,50714,50714
1,2018-01-01,5968763,2,1381,50726,50726,50731,50731
2,2018-01-01,5968763,3,1406,50773,50773,50787,50817
3,2018-01-01,5968763,4,1407,50816,50816,50871,50871
4,2018-01-01,5968763,5,1409,50855,50855,50893,50893
...,...,...,...,...,...,...,...,...
61,2018-01-01,5968763,62,467,54148,54148,53306,53306
62,2018-01-01,5968763,63,468,54205,54205,53324,53324
63,2018-01-01,5968763,64,3032,54266,54266,53413,53413
64,2018-01-01,5968763,65,3033,54339,54339,53447,53463


In [39]:
for t in trips_l:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-39-45600d9256ca>, line 2)

In [ ]:
DF_droped.describe(include="all").T

In [41]:
query = "select * from busduck.leavetimes_01 LIMIT 0,5;"
Vehicles_DF = GetData(cur, query)
DF



['DATASOURCE',
 'DAYOFSERVICE',
 'TRIPID',
 'PROGRNUMBER',
 'STOPPOINTID',
 'PLANNEDTIME_ARR',
 'PLANNEDTIME_DEP',
 'ACTUALTIME_ARR',
 'ACTUALTIME_DEP',
 'VEHICLEID',
 'PASSENGERS',
 'PASSENGERSIN',
 'PASSENGERSOUT',
 'DISTANCE',
 'SUPPRESSED',
 'JUSTIFICATIONID',
 'LASTUPDATE',
 'NOTE']